In [2]:
import boto3
import pandas as pd
import numpy as np
from pathlib import Path
import json
import re

import sys  
from saiva.model.shared.generate_base_features import processing_patient_meds, sorter_and_deduper, base_feature_processing # TODO: file does not exist
from saiva.model.shared.load_raw_data import join_tables

In [3]:
def are_dfs_equal(df1, df2):
    df = pd.concat([df1, df2]).drop_duplicates(keep=False)
    print(df.shape)
    if len(df) != 0:
        print(f"two dataframes are different!!")
    return len(df) == 0

In [4]:
s3_predictions_loc = 's3://saiva-dev-data-bucket/data/avante/2020-05-11/10/test/run1/predictions_output.parquet'
local_predictions_loc = '/data/test/2020-05-11/predictions_output.parquet'
s3_preds = pd.read_parquet(s3_predictions_loc)
local_preds = pd.read_parquet(local_predictions_loc)

In [5]:
are_dfs_equal(s3_preds, local_preds)

(76, 10)
two dataframes are different!!


False

In [6]:
s3_top15 = s3_preds.sort_values(['predictionrank']).reset_index(drop=True)['masterpatientid'][0:15]

In [7]:
local_top15 = local_preds.sort_values(['predictionrank']).reset_index(drop=True)['masterpatientid'][0:15]

In [8]:
s3_top15 == local_top15

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14    False
Name: masterpatientid, dtype: bool

In [9]:
s3_preds.sort_values(['predictionrank'])[0:16]

,masterpatientid,facilityid,bedid,censusdate,predictionvalue,predictionrank,modelid,client,experiment_group,experiment_group_rank
71,265286,10,438,2020-05-11,0.112407,1.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,1.0
99,272115,10,468,2020-05-11,0.097662,2.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,2.0
55,256828,10,547,2020-05-11,0.073306,3.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,3.0
29,188486,10,337,2020-05-11,0.062654,4.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,4.0
32,199392,10,493,2020-05-11,0.060654,5.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,5.0
88,269573,10,472,2020-05-11,0.054860,6.0,4e78363600a14e65866d8c1ef7ab28fe,avante,False,1.0
45,239122,10,500,2020-05-11,0.050398,7.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,6.0
76,266605,10,491,2020-05-11,0.049443,8.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,7.0
59,259478,10,443,2020-05-11,0.048078,9.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,8.0
46,247782,10,573,2020-05-11,0.046345,10.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,9.0


In [10]:
local_preds.sort_values(['predictionrank'])[0:16]

,masterpatientid,facilityid,bedid,censusdate,predictionvalue,predictionrank,modelid,client,experiment_group,experiment_group_rank
71,265286,10,438,2020-05-11,0.112407,1.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,1.0
99,272115,10,468,2020-05-11,0.097662,2.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,2.0
55,256828,10,547,2020-05-11,0.073306,3.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,3.0
29,188486,10,337,2020-05-11,0.063006,4.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,4.0
32,199392,10,493,2020-05-11,0.060654,5.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,5.0
88,269573,10,472,2020-05-11,0.054860,6.0,4e78363600a14e65866d8c1ef7ab28fe,avante,False,1.0
45,239122,10,500,2020-05-11,0.050398,7.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,6.0
76,266605,10,491,2020-05-11,0.049443,8.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,7.0
59,259478,10,443,2020-05-11,0.048078,9.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,8.0
46,247782,10,573,2020-05-11,0.046345,10.0,4e78363600a14e65866d8c1ef7ab28fe,avante,True,9.0
